In [19]:
import torch

/Users/elgasko/opt/anaconda3/envs/llm/lib/python3.8/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


In [29]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set the device
device = "mps" if torch.backends.mps.is_available() else "cpu"
device = torch.device(device)
print(f"Using device: {device}")


PyTorch version: 2.1.2
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


In [ ]:
# Open the txt file
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [12]:
# Inspect the .txt
print(text[:100])

The Wonderful Wizard of Oz




Chapter I
The Cyclone


Dorothy lived in the midst of the great Kans


In [13]:
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']
69


In [17]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)

print(encoded_hello, decoded_hello)

[44, 41, 48, 48, 51] hello


In [20]:
data = torch.tensor(encode(text), dtype = torch.long)

print(data[:100])

#A torch.tensor in Python, especially in the context of PyTorch, 
# is like a container for numbers or data that you can work with 
# using mathematical operations. It's similar to a list or an array,
#  but it's designed to efficiently handle large amounts of numerical data,
#  making it great for tasks like machine learning and deep learning. 
# You can perform operations like addition, multiplication,
#  and more on torch.tensor objects, which makes it useful for building 
# and training neural networks.

tensor([68, 30, 44, 41,  1, 33, 51, 50, 40, 41, 54, 42, 57, 48,  1, 33, 45, 62,
        37, 54, 40,  1, 51, 42,  1, 25, 62,  0,  0,  0,  0,  0, 13, 44, 37, 52,
        56, 41, 54,  1, 19,  0, 30, 44, 41,  1, 13, 61, 39, 48, 51, 50, 41,  0,
         0,  0, 14, 51, 54, 51, 56, 44, 61,  1, 48, 45, 58, 41, 40,  1, 45, 50,
         1, 56, 44, 41,  1, 49, 45, 40, 55, 56,  1, 51, 42,  1, 56, 44, 41,  1,
        43, 54, 41, 37, 56,  1, 21, 37, 50, 55])


In [21]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [26]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print('when input is ', context, 'target is ', target)

print('example tensor is ', x)

when input is  tensor([68]) target is  tensor(30)
when input is  tensor([68, 30]) target is  tensor(44)
when input is  tensor([68, 30, 44]) target is  tensor(41)
when input is  tensor([68, 30, 44, 41]) target is  tensor(1)
when input is  tensor([68, 30, 44, 41,  1]) target is  tensor(33)
when input is  tensor([68, 30, 44, 41,  1, 33]) target is  tensor(51)
when input is  tensor([68, 30, 44, 41,  1, 33, 51]) target is  tensor(50)
when input is  tensor([68, 30, 44, 41,  1, 33, 51, 50]) target is  tensor(40)
example tensor is  tensor([68, 30, 44, 41,  1, 33, 51, 50])
